# <span style="color:lightblue"> DATA SCIENCE AND MACHINE LEARNING INTERNSHIP PROGRAM </span>

<h2><center><u>Assignment_4</u></center><h2>

In [1]:
# Importing useful libraries

#dataFrame manipulation and visualiztion
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('Display.max_columns', None)
pd.set_option('Display.max_rows', None)

#Needed libraries
import datetime

## <span style="color:lightblue"> **Problem Statement 1:** </span>
The data (sample) were collected in São Paulo - Brazil, in a university, where there are some parties with groups of students from 18 to 28 years of age (average). The dataset used for this activity has 7 attributes, being a Target, with a period of one year. You have to predict the quantity of beer consumption based on the features that contain climate conditions.

**Dataset Description**
- Data: date of the record
- Temperatura Media (C): Average temperature of the day in celsius
- Temperatura Minima (C): Minimum temperature of the day in celsius
- Temperatura Maxima (C): Maximum temperature of the day in celsius
- Precipitacao (mm): Percipitation in mm
- Final de Semana: If the day is the weekend or not
- Consumo de cerveja (litros): Beer consumption in liters

I would be writing python code to perform the following tasks mentioned:
1. Load the dataset, check its shape
2. Rectify the data of the first four columns
   **Hint:** Check columns 'Temperatura Media(C)', 'Temperatura Minima (C)', 'Temperatura Maxima (C)' and 'Precipitacao'<br>
   Fix the following errors present in these features
3. Create new features using the 'Data' feature and the make 'Data' column as index<br>
    **Hint:** Create a new feature 'Month' from the dates, consisting of the month of the year.<br>
    Create a new feature 'Day' from the dates, consisting of the day of the week.
    Set values from the 'Data' column as indexes. Use code snippet:
    `df1.set_index('Data', inplace=True)`
4. Handle null and duplicate values
5. Check the data type of the features and convert them to the appropriate data type
6. Analyse features with outlier values
7. Plot and analyze the correlation
8. Split the datast for training and testing
9. Train a linear regression model and print the intercept and coefficients
10. Evaluate the model using the R2 score, mean absolute error and root mean squared error


#### Beer_df refers to the dataframe for the Consumo_cervea data

In [2]:
beer_df = pd.read_csv("./../Assignment_files/Consumo_cerveja_ass4.csv")
beer_df.shape

(941, 7)

#### creating a date column from the data column of dtype datetime so I can access the dates

In [3]:
date_df = pd.to_datetime(beer_df["Data"])
beer_df.head()

,Data,Temperatura Media (C),Temperatura Minima (C),Temperatura Maxima (C),Precipitacao (mm),Final de Semana,Consumo de cerveja (litros)
0,2015-01-01,"27,3","23,9","32,5",0,0.0,25.461
1,2015-01-02,"27,02","24,5","33,5",0,0.0,28.972
2,2015-01-03,"24,82","22,4","29,9",0,1.0,30.814
3,2015-01-04,"23,98","21,5","28,6","1,2",1.0,29.799
4,2015-01-05,"23,82",21,"28,3",0,0.0,28.900


#### Converting the object columns of dtype to floats

In [4]:
for column in beer_df.columns:
    if beer_df[column].dtype == 'object':
        beer_df[column] = beer_df[column].str.replace(",", ".")

#convert columns to numeric dtypes
beer_df = beer_df.apply(pd.to_numeric, errors='coerce')

#convert data to datetime object to be able to extract days, months, time in general
beer_df['Data'] = date_df

In [5]:
beer_df.dtypes

Data                           datetime64[ns]
Temperatura Media (C)                 float64
Temperatura Minima (C)                float64
Temperatura Maxima (C)                float64
Precipitacao (mm)                     float64
Final de Semana                       float64
Consumo de cerveja (litros)           float64
dtype: object

In [6]:
beer_df.set_index('Data', inplace=True)
beer_df.head()

,Temperatura Media (C),Temperatura Minima (C),Temperatura Maxima (C),Precipitacao (mm),Final de Semana,Consumo de cerveja (litros)
Data,,,,,,
2015-01-01,27.30,23.9,32.5,0.0,0.0,25.461
2015-01-02,27.02,24.5,33.5,0.0,0.0,28.972
2015-01-03,24.82,22.4,29.9,0.0,1.0,30.814
2015-01-04,23.98,21.5,28.6,1.2,1.0,29.799
2015-01-05,23.82,21.0,28.3,0.0,0.0,28.900


In [7]:
# Creating new features
beer_df['Month'] = pd.DatetimeIndex(date_df).month
beer_df['Day'] = pd.DatetimeIndex(date_df).day_of_week  # The day of the week with Monday=0, Sunday=6
beer_df.head()

,Temperatura Media (C),Temperatura Minima (C),Temperatura Maxima (C),Precipitacao (mm),Final de Semana,Consumo de cerveja (litros),Month,Day
Data,,,,,,,,
2015-01-01,27.30,23.9,32.5,0.0,0.0,25.461,1.0,3.0
2015-01-02,27.02,24.5,33.5,0.0,0.0,28.972,1.0,4.0
2015-01-03,24.82,22.4,29.9,0.0,1.0,30.814,1.0,5.0
2015-01-04,23.98,21.5,28.6,1.2,1.0,29.799,1.0,6.0
2015-01-05,23.82,21.0,28.3,0.0,0.0,28.900,1.0,0.0


In [8]:
beer_df.dtypes

Temperatura Media (C)          float64
Temperatura Minima (C)         float64
Temperatura Maxima (C)         float64
Precipitacao (mm)              float64
Final de Semana                float64
Consumo de cerveja (litros)    float64
Month                          float64
Day                            float64
dtype: object

In [12]:
beer_df.duplicated().sum(), len(beer_df)

(575, 941)

More than half of the data is duplicated 

In [19]:
beer_df.drop_duplicates(inplace=True)
beer_df.duplicated().sum()

0

In [21]:
beer_df.isna().sum()

Temperatura Media (C)          1
Temperatura Minima (C)         1
Temperatura Maxima (C)         1
Precipitacao (mm)              1
Final de Semana                1
Consumo de cerveja (litros)    1
Month                          1
Day                            1
dtype: int64

After removing duplicate entries I would fill up the missing values with the mean value for temperature columns and mode value for datatime columns

## <span style="color:lightblue"> **Problem Statement 2:** </span>
I am provided with the california housing dataset. Based on the given parameters of a house, predict its price.

**Dataset Description:**

The dataset contains nine features:
- Longitude: A measure of how far west a house is; a higher value is farther west
- Latitude: A measure of how far north a house is; a higher value is farther north
- hosingMedianAge: Meduan age of a house within a block; a lower number is a newser building
- total rooms: Total number of rooms withina block
- total bedrooms: Total number of bedrooms within a block
- populations: Total number of people residing within a block
- households: Total number of households, a group of people residing within a home unit, for a block
- median income: Median income for households within a block of houses (measured in tens of thousands of US Dollars)
- median house value: Median house value for households within a block (measured in US Dollars)

I'm writing a python code to perform the following tasks mentioned:
1. Load the datam check its shape and check for null values
2. Split the datset for training and testing - 1000- instances for testing
3. Train the model using sklearn (Apply linear regression to train a model for prediction)
4. Predict the prices on test data and evaluate the model by r2 score and mean absolute error
5. Find coefficient and intercept using the trained model

In [ ]:
beer_df.head()

## <span style="color:lightblue"> **Problem Statement 3:** </span>
I have been provided with the  medical cost dataset. I need to predict individual medical costs billed by health insurance.

Dataset Description:
- age: age of the primary beneficiary
- sex: gender of primary beneficiary female, male
- bmi: Body mass index, providing an understanding of the body, weights that are relatively<br>
  high or low relative to height, an objective index of body.
- weight(kg/m ^ 2) using the ratio of height to weight, ideally 18.5 to 24.9
- Children: Number of children covered by health insurance / Number of dependents
- smoker: Smokes or not
- region: the beneficiary's residential area in the US, northest, southeast, southwest, northwest
- charges: Individual medical costs billed by health insurance

I would write a python code to perform the following tasks mentioned:
1. Load the datam check its shape and check for null values
2. Convert categorical features to numerical values (Use One-Hot Encoding)
3. Split the dataset for training and testing
4. Train the model using sklearn - Linear Regression
5. Find the intercept and coefficient from the trained model
6. Predict the prices of test data and evalutate the model using calculated r2 score and root mean squared error

## <span style="color:lightblue"> **Problem Statement 4:** </span>
I have been provided with the '50_Startups' data. Using the given features, I must predict the profit of these startups.

**Dataset Description:**
- R&D Spend: Expenditures in Research and Development
- Administration: Expenditures in Administration
- Marketing Spend: Expenditures in Marketing
- State: In which state the company belongs to
- Profit: The profit made by the company

I would write python code to perform the following tasks mentioned:
1. Load the data, check its shape and check for null values
2. Convert categorical features to numerical values using Label Encoder
3. Split the dataset for training and testing
4. Train the model using sklearn (linear regression), also find the intercept and coefficient from the trained model
5. Predict the profits of test data and evaluate the model using r2 score and mean squared error
6. Regularize the model using Ridge Regression and find the score
7. Regularize the model using Lasso Regeression and find the score